# Baseline Models
A baseline model can be defined as a simple model that provides reasonable results on a task and requires not much time and expertise to develop. Baseline models helps put the more complex models into context in terms of accuracy. The results obtained from a baseline model should guide us in making the choice of complex model to use. In this classification task, we will use `Linear Support Vector Classifier`, `Support Vector Classifier`, `Multilayer Perceptron Classifier` and `Random Forests Classifiers` as our baseline models.

In [ ]:
#import necessary libraries
import os
import random
import warnings
import numpy as np
import pandas as pd
import configparser
from tqdm import tqdm
import tensorflow as tf
from keras import layers
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

## Generate and save various parameters needed for the baseline models

In [ ]:
if not os.path.exists('./parameters'):
    os.makedirs('./parameters')

np.save('./parameters/svm.npy', np.linspace(10e-2,10e2,200)) #C parameters for svm.LinearSVC and svm.SVC
np.save('./parameters/svm-rbf.npy', np.linspace(10e-2,10e2,200)) #C parameters for svm.SVC
np.save('./parameters/mlp.npy', np.arange(100,510, 10)) #hidden_layer_sizes for mlp classifier
np.save('./parameters/rf.npy', np.arange(200, 2000, 200)) #n_estimators for Random Forest Classifier


warnings.filterwarnings('ignore')

## Data preprocessing
Acoustic classification of bird species is often a challenging task due to some of the following reasons:
<ol>
<li>Background noise</li>
<li>Variable length of sound recordings</li>
<li>Few number of annotated recordings per species</li>
</ol>

For this task, we used various data preprocessing and data augmentation methods to tackle these problems. For background noise, we began with separating audio files into signal part (sections with a bird call/song) and noise part (parts with no bird call/song and that contain background noise). For the variable length of sound recordings, we split the audio into fixed size chunks to be fed to the models. Audio files that were less than the required fixed length were padded with noise to make them to be of at least the required fixed size. This in turn helped in increasing the number of inputs into the classifiers. From a single file, several chunks were obtained hence increasing the number of inputs to the classifiers. 

In [ ]:
def compute_feature_mean_std(file_list):
    """Compute the mean and standard deviation of all the features
    in file_list for use in normalisation
    Args:
        file_list: list of complete path to file to get features from
    Returns:
        mean: mean of all channels
        std: standard deviation of all channels
    """
    all_feature = np.array([])

    for filename in file_list:

        curr_feature = np.load(filename)
        curr_feature = np.log(curr_feature + 1e-8)


        if all_feature.size:
            all_feature = np.vstack((all_feature, curr_feature.T))
        else:
            all_feature = curr_feature.T


    return np.mean(all_feature, axis=0), np.std(all_feature, axis=0)


def all_summary_features(file_list, annotation_dict, fmean, fstd, num_frame=10):

    features = []
    labels = []

    for filename in file_list:



        feature = np.load(filename)
        feature = np.log(feature + 1e-8)

        feature = ((feature.T - fmean) /
                       (fstd + 1e-8)).T

        if feature.shape[1] > 2 * num_frame + 1:

            for indx in range(num_frame, feature.shape[1] - num_frame - 1, num_frame):

                current_feature = feature[:, indx - num_frame: indx + num_frame + 1]


                features.append(np.concatenate((np.mean(current_feature, axis=1),
                                                np.std(current_feature, axis=1))))


                labels.append(annotation_dict[filename])

    return np.array(features), labels

## Models

In [ ]:


def main(mels_dir, annotation_csv, model=None, param_dir=None):
    
    
    # Get parameters from configuration file
    config = configparser.ConfigParser()
    config.read('parameters.ini')

    win_len_ms = int(config['audio']['win_len_ms'])
    overlap = float(config['audio']['overlap'])
    sampling_rate = int(config['audio']['sampling_rate'])
    duration = float(config['neural-net']['input_duration_s'])
    rnd_seed = int(config['neural-net']['seed'])


    # Derive audio processing values
    win_len = int((win_len_ms * sampling_rate) / 1000)
    hop_len = int(win_len * (1 - overlap))
    
    np.random.seed(rnd_seed)
    random.seed(rnd_seed)
    tf.random.set_seed(rnd_seed)
    labels = []
    df_annotation = pd.read_csv(annotation_csv)
    species = list(set(df_annotation['label']))
    species.sort()
    species_dict = dict(zip(species, range(len(species))))
    for file_species in list(df_annotation['label']):
        labels.append(species_dict[file_species])
    
    filelist = list(df_annotation.name)
    for indx, filename in enumerate(filelist):
        filelist[indx] = os.path.join(mels_dir, filename)
    
    annotation_dict = dict(zip(filelist, labels))
    
    training_files, validation_files = train_test_split(filelist, test_size=.3)
    feature_mean, feature_std = compute_feature_mean_std(training_files[:])
    num_frame = int((0.5 * duration * sampling_rate) / hop_len)
    
    X_train, y_train = all_summary_features(training_files,
                                               annotation_dict,
                                               feature_mean,
                                               feature_std,
                                               num_frame)
    X_val, y_val = all_summary_features(validation_files,
                                           annotation_dict,
                                           feature_mean,
                                           feature_std,
                                           num_frame)
    
    
    param_file = os.path.join(param_dir, model + '.npy')
    parameters = np.load(param_file, allow_pickle=True)
    
    if model == 'svm':
        clf = LinearSVC(random_state=0, tol=1e-5, multi_class='crammer_singer', max_iter=10000)
    elif model == 'svm-rbf':
        clf = SVC(gamma='auto')
    elif model == 'mlp':
        clf = MLPClassifier(random_state=1, max_iter=10000)
    elif model == 'rf':
        clf = RandomForestClassifier(max_depth=None, random_state=0)
        
        
    val_accuracy = []
    mean_f1_score = []
    
    params = tqdm(parameters)
    for param in params:
        if model == 'svm':
            params.set_description("C = %s" % param)
            clf.set_params(C=param)
            
        if model == 'svm-rbf':
            params.set_description("C = %s" % param)
            clf.set_params(C=param)

        if model == 'mlp':
            params.set_description("hidden_layer_sizes = %s" % param)
            clf.set_params(hidden_layer_sizes=param)

        if model == 'rf':
            params.set_description("n_estimators=param = %s" % param)
            clf.set_params(n_estimators=param)

        clf.fit(X_train, y_train)
        val_accuracy.append(sum(clf.predict(X_val) == y_val) / X_val.shape[0])
        mean_f1_score.append(np.mean(f1_score(y_val, clf.predict(X_val), average=None)))
        
        
        
    best_param = parameters[np.argmax(mean_f1_score)]

    if 'svm' in model:
        clf.set_params(C=best_param)

    if model == 'mlp':
        clf.set_params(hidden_layer_sizes=best_param)

    if model == 'rf':
        clf.set_params(n_estimators=best_param)

    clf.fit(X_train, y_train)
    
    
    species_f1_score = f1_score(y_val, clf.predict(X_val), average=None)
    species_precision_score = precision_score(y_val, clf.predict(X_val), average=None)
    species_recall_score = recall_score(y_val, clf.predict(X_val), average=None)
    df_species_metrics = pd.DataFrame(list(zip(species,
                                               species_precision_score,
                                               species_recall_score,
                                               species_f1_score)),
                                      columns=['Species', 'Precision', 'Recall', 'F1 Score'])
    df_species_metrics = df_species_metrics.sort_values(['F1 Score'], ascending=False)
    df_species_metrics = df_species_metrics.reset_index(drop=True)

    print('Mean F1 score:', np.mean(species_f1_score))
    print(df_species_metrics.round(2))
    
    
    

### Random Forest Classifier

In [ ]:
if __name__ == '__main__':
    main('./melspectrograms',
         'labels.csv',
         model='rf',
         param_dir='parameters')

### Multilayer Perceptron Classifier

In [ ]:
if __name__ == '__main__':
    main('./melspectrograms',
         'labels.csv',
         model='mlp',
         param_dir='parameters')

### Linear Support Vector Classifier

In [ ]:
if __name__ == '__main__':
    main('./melspectrograms',
         'labels.csv',
         model='svm',
         param_dir='parameters')

### Support Vector Classifier

In [ ]:
if __name__ == '__main__':
    main('./melspectrograms',
         'labels.csv',
         model='svm-rbf',
         param_dir='parameters')
    